<a href="https://colab.research.google.com/github/niger222/tkinter-student-display/blob/main/kociemba.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kociemba
!pip install pdf2image
!apt-get install poppler-utils
!which pdftoppm
!pip install PyMuPDF

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 111.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for kociemba: filename=kociemba-1.2.1-cp311-cp311-linux_x86_64.whl size=6800268 sha256=bbb52a8790f594c066b56948751afad02a0dec292ccdb7b6ddf3b2e2123750e0
  Stored in directory: /root/.cache/pip/wheels/6c/51/2f/f3b8548d55efe500bd3b8880b0c59e7c59d0bf765c5676c036
Successfully built kociemba
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 34 not upgraded.
Need to get 186 kB of archives.
After this operation, 696 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.7 [186 kB]
Fetched 186 kB in 2s (117 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 126333 files and directories cur

In [ ]:
import os
import cv2
import numpy as np
import fitz
import matplotlib.pyplot as plt
from google.colab import drive
from scipy.optimize import linear_sum_assignment
from collections import Counter
drive.mount('/content/drive')
training_folder = "/content/drive/MyDrive/numpy project/"# 訓練資料夾路徑

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **顏色數據統計**

1.   由於雲端有內建掃描功能，所以圖片都用掃描的呈現能增加準確度以及節省時間，但用掃描的缺點在於會變成pdf檔案


*   因此使用`pdf_to_cv2_image`函式來轉換檔案

2.   由於圖像拍攝過程中的角度、光線等因素，同一顏色在不同位置的` HSV `值會有所差異。


*   因此使用`aggregate_all_cell_hsv`函式將每個格子內的` HSV `數據記錄到`numpy`陣列裡面，這樣可以為後續的顏色提供更準確的數據，提高顏色辨識的成功率。

In [ ]:
import os
import cv2
import numpy as np
import fitz
import matplotlib.pyplot as plt
from google.colab import drive
from scipy.optimize import linear_sum_assignment
from collections import Counter
drive.mount('/content/drive')
training_folder = "/content/drive/MyDrive/numpy project/"# 訓練資料夾路徑
def pdf_to_cv2_image(pdf_path, zoom=2):# 使用PyMuPDF將PDF檔案轉換成影像（只取第一頁），並轉換成OpenCV可用的BGR格式
    doc = fitz.open(pdf_path)  # 開啟 PDF 檔案
    page = doc.load_page(0)    # 載入 PDF 檔案的第一頁
    mat = fitz.Matrix(zoom, zoom)  # 設定縮放比率，增大圖片解析度
    pix = page.get_pixmap(matrix=mat)  # 將頁面轉換為像素圖（pixmap）
    img = np.frombuffer(pix.samples, dtype=np.uint8).reshape(pix.height, pix.width, pix.n) # 將轉換過來的像素資料從buffer轉為numpy陣列
    # 根據pixmap的通道數來判斷顏色空間格式
    if pix.n == 4: # 如果有alpha通道轉為BGR格式
        img = cv2.cvtColor(img, cv2.COLOR_BGRA2BGR)
    else: # 如果是RGB則轉為BGR格式
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    return img

def aggregate_all_cell_hsv(training_folder):# 從訓練資料夾讀取並聚合每個面所對應的所有格子的HSV數據
    faces = ['U', 'R', 'L', 'F', 'B', 'D']  # 魔術方塊的 6 個面
    training_records = {}  # 儲存每個面的所有HSV數據
    file_extensions = ('.pdf', '.png', '.jpg', '.jpeg')
    for face in faces:  # 遍歷每個面
        face_folder = os.path.join(training_folder, face)  # 指定該面所在的資料夾
        files = sorted([os.path.join(face_folder, f) for f in os.listdir(face_folder)if f.lower().endswith(file_extensions)]) # 讀取資料夾內所有圖片檔案
        cell_hsv_list = []  # 儲存每張圖片的每個格子的 HSV 數值
        for file in files:
            ext = os.path.splitext(file)[1].lower()  # 取得檔案的副檔名
            if ext == ".pdf":
                img = pdf_to_cv2_image(file)  # 如果是PDF將其轉換為影像
            else:
                img = cv2.imread(file)  # 否則直接讀取圖片檔案
            hsv_img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)# 將BGR影像轉換成HSV色彩空間
            h, w, _ = hsv_img.shape  # 取得影像的高度和寬度
            step_h = h // 3  # 每個格子在垂直方向的像素高度
            step_w = w // 3  # 每個格子在水平方向的像素寬度
            # 遍歷每個 3×3 格子，計算每個格子的平均 HSV 值
            for i in range(3):
                for j in range(3):
                    square = hsv_img[i * step_h : (i + 1) * step_h, j * step_w : (j + 1) * step_w]# 計算當前格子內所有像素的平均HSV值
                    avg_hsv = np.mean(square.reshape(-1, 3), axis=0)
                    cell_hsv_list.append(avg_hsv)  # 將每個格子的平均HSV值儲存
        training_records[face] = cell_hsv_list# 將該面所有格子的HSV數據儲存到字典中
    return training_records

training_records = aggregate_all_cell_hsv(training_folder)
# 顯示所有面中各格子的 HSV 數值
for face, hsv_values in training_records.items():
    print(f"Face {face}:")
    for idx, hsv in enumerate(hsv_values):
        print(f"  Cell {idx + 1}: {hsv}")

NameError: name 'training_folder' is not defined

# **匯入要判斷的魔術方塊**


In [ ]:
import os
import cv2
import numpy as np
import fitz
import matplotlib.pyplot as plt
from google.colab import drive
from scipy.optimize import linear_sum_assignment
from collections import Counter
drive.mount('/content/drive')
training_folder = "/content/drive/MyDrive/numpy project/"# 訓練資料夾路徑
def pdf_to_cv2_image(pdf_path, zoom=2):# 使用PyMuPDF將PDF檔案轉換成影像（只取第一頁），並轉換成OpenCV可用的BGR格式
    doc = fitz.open(pdf_path)  # 開啟 PDF 檔案
    page = doc.load_page(0)    # 載入 PDF 檔案的第一頁
    mat = fitz.Matrix(zoom, zoom)  # 設定縮放比率，增大圖片解析度
    pix = page.get_pixmap(matrix=mat)  # 將頁面轉換為像素圖（pixmap）
    img = np.frombuffer(pix.samples, dtype=np.uint8).reshape(pix.height, pix.width, pix.n) # 將轉換過來的像素資料從buffer轉為numpy陣列
    # 根據pixmap的通道數來判斷顏色空間格式
    if pix.n == 4: # 如果有alpha通道轉為BGR格式
        img = cv2.cvtColor(img, cv2.COLOR_BGRA2BGR)
    else: # 如果是RGB則轉為BGR格式
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    return img

def aggregate_all_cell_hsv(training_folder):# 從訓練資料夾讀取並聚合每個面所對應的所有格子的HSV數據
    faces = ['U', 'R', 'L', 'F', 'B', 'D']  # 魔術方塊的 6 個面
    training_records = {}  # 儲存每個面的所有HSV數據
    file_extensions = ('.pdf', '.png', '.jpg', '.jpeg')
    for face in faces:  # 遍歷每個面
        face_folder = os.path.join(training_folder, face)  # 指定該面所在的資料夾
        files = sorted([os.path.join(face_folder, f) for f in os.listdir(face_folder)if f.lower().endswith(file_extensions)]) # 讀取資料夾內所有圖片檔案
        cell_hsv_list = []  # 儲存每張圖片的每個格子的 HSV 數值
        for file in files:
            ext = os.path.splitext(file)[1].lower()  # 取得檔案的副檔名
            if ext == ".pdf":
                img = pdf_to_cv2_image(file)  # 如果是PDF將其轉換為影像
            else:
                img = cv2.imread(file)  # 否則直接讀取圖片檔案
            hsv_img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)# 將BGR影像轉換成HSV色彩空間
            h, w, _ = hsv_img.shape  # 取得影像的高度和寬度
            step_h = h // 3  # 每個格子在垂直方向的像素高度
            step_w = w // 3  # 每個格子在水平方向的像素寬度
            # 遍歷每個 3×3 格子，計算每個格子的平均 HSV 值
            for i in range(3):
                for j in range(3):
                    square = hsv_img[i * step_h : (i + 1) * step_h, j * step_w : (j + 1) * step_w]# 計算當前格子內所有像素的平均HSV值
                    avg_hsv = np.mean(square.reshape(-1, 3), axis=0)
                    cell_hsv_list.append(avg_hsv)  # 將每個格子的平均HSV值儲存
        training_records[face] = cell_hsv_list# 將該面所有格子的HSV數據儲存到字典中
    return training_records

training_records = aggregate_all_cell_hsv(training_folder)
# 顯示所有面中各格子的 HSV 數值
for face, hsv_values in training_records.items():
    print(f"Face {face}:")
    for idx, hsv in enumerate(hsv_values):
        print(f"  Cell {idx + 1}: {hsv}")
folder_path = "/content/drive/MyDrive/numpy project/test/"  # 換成讀取test裡的資料
file_extensions = ('.pdf', '.png', '.jpg', '.jpeg')
test_files = sorted([os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.lower().endswith(file_extensions)])  # 讀取所有符合條件的檔案
images = []  # 用來儲存轉換後的影像

for file_path in test_files: # 讀取每個檔案並轉換RGB
    ext = os.path.splitext(file_path)[1].lower()
    if ext == ".pdf":
        img = pdf_to_cv2_image(file_path)
    else:
        img = cv2.imread(file_path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    images.append(img_rgb)  # 將處理後的影像加入列表

print(f"總共讀到 {len(images)} 張影像 (轉換自 PDF 或圖片)。")

總共讀到 6 張影像 (轉換自 PDF 或圖片)。


# **數據分析**

這段程式的主要目的是將魔術方塊的影像資料進行處理，計算每個面所有區域的 HSV 值，並根據之前的訓練數據來判斷每個區塊的顏色。



1. 計算所有TEST資料的HSV值。
2. 與之前訓練出的所有cell做比對
3. 固定每一面的中心顏色
4. 因所有顏色都只有九個，所以這個步驟是取最接近的九個，使用`Hungarian`演算法求解
5. 將結果組合成 Cube String供kociemba使用

In [ ]:
import os
import cv2
import numpy as np
import fitz
import matplotlib.pyplot as plt
from google.colab import drive
from scipy.optimize import linear_sum_assignment
from collections import Counter
drive.mount('/content/drive')
training_folder = "/content/drive/MyDrive/numpy project/"# 訓練資料夾路徑
def pdf_to_cv2_image(pdf_path, zoom=2):# 使用PyMuPDF將PDF檔案轉換成影像（只取第一頁），並轉換成OpenCV可用的BGR格式
    doc = fitz.open(pdf_path)  # 開啟 PDF 檔案
    page = doc.load_page(0)    # 載入 PDF 檔案的第一頁
    mat = fitz.Matrix(zoom, zoom)  # 設定縮放比率，增大圖片解析度
    pix = page.get_pixmap(matrix=mat)  # 將頁面轉換為像素圖（pixmap）
    img = np.frombuffer(pix.samples, dtype=np.uint8).reshape(pix.height, pix.width, pix.n) # 將轉換過來的像素資料從buffer轉為numpy陣列
    # 根據pixmap的通道數來判斷顏色空間格式
    if pix.n == 4: # 如果有alpha通道轉為BGR格式
        img = cv2.cvtColor(img, cv2.COLOR_BGRA2BGR)
    else: # 如果是RGB則轉為BGR格式
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    return img

def aggregate_all_cell_hsv(training_folder):# 從訓練資料夾讀取並聚合每個面所對應的所有格子的HSV數據
    faces = ['U', 'R', 'L', 'F', 'B', 'D']  # 魔術方塊的 6 個面
    training_records = {}  # 儲存每個面的所有HSV數據
    file_extensions = ('.pdf', '.png', '.jpg', '.jpeg')
    for face in faces:  # 遍歷每個面
        face_folder = os.path.join(training_folder, face)  # 指定該面所在的資料夾
        files = sorted([os.path.join(face_folder, f) for f in os.listdir(face_folder)if f.lower().endswith(file_extensions)]) # 讀取資料夾內所有圖片檔案
        cell_hsv_list = []  # 儲存每張圖片的每個格子的 HSV 數值
        for file in files:
            ext = os.path.splitext(file)[1].lower()  # 取得檔案的副檔名
            if ext == ".pdf":
                img = pdf_to_cv2_image(file)  # 如果是PDF將其轉換為影像
            else:
                img = cv2.imread(file)  # 否則直接讀取圖片檔案
            hsv_img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)# 將BGR影像轉換成HSV色彩空間
            h, w, _ = hsv_img.shape  # 取得影像的高度和寬度
            step_h = h // 3  # 每個格子在垂直方向的像素高度
            step_w = w // 3  # 每個格子在水平方向的像素寬度
            # 遍歷每個 3×3 格子，計算每個格子的平均 HSV 值
            for i in range(3):
                for j in range(3):
                    square = hsv_img[i * step_h : (i + 1) * step_h, j * step_w : (j + 1) * step_w]# 計算當前格子內所有像素的平均HSV值
                    avg_hsv = np.mean(square.reshape(-1, 3), axis=0)
                    cell_hsv_list.append(avg_hsv)  # 將每個格子的平均HSV值儲存
        training_records[face] = cell_hsv_list# 將該面所有格子的HSV數據儲存到字典中
    return training_records

training_records = aggregate_all_cell_hsv(training_folder)
# 顯示所有面中各格子的 HSV 數值
for face, hsv_values in training_records.items():
    print(f"Face {face}:")
    for idx, hsv in enumerate(hsv_values):
        print(f"  Cell {idx + 1}: {hsv}")
folder_path = "/content/drive/MyDrive/numpy project/test/"  # 換成讀取test裡的資料
file_extensions = ('.pdf', '.png', '.jpg', '.jpeg')
test_files = sorted([os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.lower().endswith(file_extensions)])  # 讀取所有符合條件的檔案
images = []  # 用來儲存轉換後的影像

for file_path in test_files: # 讀取每個檔案並轉換RGB
    ext = os.path.splitext(file_path)[1].lower()
    if ext == ".pdf":
        img = pdf_to_cv2_image(file_path)
    else:
        img = cv2.imread(file_path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    images.append(img_rgb)  # 將處理後的影像加入列表

print(f"總共讀到 {len(images)} 張影像 (轉換自 PDF 或圖片)。")
#步驟1
def compute_face_cell_avgs(image):
    hsv_img = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
    h, w, _ = hsv_img.shape
    step_h = h // 3
    step_w = w // 3
    avgs = []
    for i in range(3):# 判斷從左到右、從上到下
        for j in range(3):
            square = hsv_img[i*step_h:(i+1)*step_h, j*step_w:(j+1)*step_w]#所有面的像素數據
            avg = np.mean(square.reshape(-1, 3), axis=0)#計算區塊的平均hsv值
            avgs.append(avg)
    return np.array(avgs)

# 計算所有54格的HSV平均值，並合併為陣列
cell_avgs_all = np.vstack([compute_face_cell_avgs(image) for image in images])
# 步驟2
color_order = ['U', 'R', 'F', 'D', 'L', 'B']
n_cells = cell_avgs_all.shape[0]#格子數量
n_colors = len(color_order)#顏色數量
cost = np.zeros((n_cells, n_colors))#儲存TEST與Train的相似度
for i in range(n_cells):
    for j, color in enumerate(color_order):
        #計算TEST與Train的最小歐式距離
        cost[i, j] = np.min(np.linalg.norm(cell_avgs_all[i] - training_records[color], axis=1))
#步驟3
fixed_indices = [4, 13, 22, 31, 40, 49]
fixed_labels = {4: 'U', 13: 'R', 22: 'F', 31: 'D', 40: 'L', 49: 'B'}
for idx in fixed_indices:
    expected = fixed_labels[idx]#預期顏色
    for j, color in enumerate(color_order):
        if color == expected:
            cost[idx, j] = 0
        else:
            cost[idx, j] = 1e6
#步驟4
#使用 Hungarian 演算法解分配問題
row_ind, col_ind = linear_sum_assignment(np.hstack([np.tile(cost[:, j:j+1], (1, 9)) for j in range(n_colors)]))
assignment = [color_order[col // 9] for col in col_ind]#將每個格子的顏色數據儲存到assignment裡面形成最終列表

# 步驟5
cube_faces_assigned = [assignment[i*9:(i+1)*9] for i in range(6)]
cube_string = "".join(sum(cube_faces_assigned, []))
print("Cube String:", cube_string)
print("各顏色出現次數:", Counter(cube_string))

Cube String: DURUUUUUURRBRRDRRDLFFFFFFFFDDDDDDLLLRBBRLLFLLULBBBBBBU
各顏色出現次數: Counter({'D': 9, 'U': 9, 'R': 9, 'B': 9, 'L': 9, 'F': 9})


# **魔術方塊解**
1.   `cube_string`必須依照`kociemba`要求的面順序`U, R, F, D, L, B`
2.   `cube_string`一定要有54個字元
3.   使用方式
*   R -> 右
*   L -> 左
*   U -> 上
*   D -> 下
*   F -> 前
*   B -> 後
*   ' -> 逆
*   2 -> 轉兩次

In [ ]:
import os
import cv2
import numpy as np
import fitz
import matplotlib.pyplot as plt
from google.colab import drive
from scipy.optimize import linear_sum_assignment
from collections import Counter
drive.mount('/content/drive')
training_folder = "/content/drive/MyDrive/numpy project/"# 訓練資料夾路徑
def pdf_to_cv2_image(pdf_path, zoom=2):# 使用PyMuPDF將PDF檔案轉換成影像（只取第一頁），並轉換成OpenCV可用的BGR格式
    doc = fitz.open(pdf_path)  # 開啟 PDF 檔案
    page = doc.load_page(0)    # 載入 PDF 檔案的第一頁
    mat = fitz.Matrix(zoom, zoom)  # 設定縮放比率，增大圖片解析度
    pix = page.get_pixmap(matrix=mat)  # 將頁面轉換為像素圖（pixmap）
    img = np.frombuffer(pix.samples, dtype=np.uint8).reshape(pix.height, pix.width, pix.n) # 將轉換過來的像素資料從buffer轉為numpy陣列
    # 根據pixmap的通道數來判斷顏色空間格式
    if pix.n == 4: # 如果有alpha通道轉為BGR格式
        img = cv2.cvtColor(img, cv2.COLOR_BGRA2BGR)
    else: # 如果是RGB則轉為BGR格式
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    return img

def aggregate_all_cell_hsv(training_folder):# 從訓練資料夾讀取並聚合每個面所對應的所有格子的HSV數據
    faces = ['U', 'R', 'L', 'F', 'B', 'D']  # 魔術方塊的 6 個面
    training_records = {}  # 儲存每個面的所有HSV數據
    file_extensions = ('.pdf', '.png', '.jpg', '.jpeg')
    for face in faces:  # 遍歷每個面
        face_folder = os.path.join(training_folder, face)  # 指定該面所在的資料夾
        files = sorted([os.path.join(face_folder, f) for f in os.listdir(face_folder)if f.lower().endswith(file_extensions)]) # 讀取資料夾內所有圖片檔案
        cell_hsv_list = []  # 儲存每張圖片的每個格子的 HSV 數值
        for file in files:
            ext = os.path.splitext(file)[1].lower()  # 取得檔案的副檔名
            if ext == ".pdf":
                img = pdf_to_cv2_image(file)  # 如果是PDF將其轉換為影像
            else:
                img = cv2.imread(file)  # 否則直接讀取圖片檔案
            hsv_img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)# 將BGR影像轉換成HSV色彩空間
            h, w, _ = hsv_img.shape  # 取得影像的高度和寬度
            step_h = h // 3  # 每個格子在垂直方向的像素高度
            step_w = w // 3  # 每個格子在水平方向的像素寬度
            # 遍歷每個 3×3 格子，計算每個格子的平均 HSV 值
            for i in range(3):
                for j in range(3):
                    square = hsv_img[i * step_h : (i + 1) * step_h, j * step_w : (j + 1) * step_w]# 計算當前格子內所有像素的平均HSV值
                    avg_hsv = np.mean(square.reshape(-1, 3), axis=0)
                    cell_hsv_list.append(avg_hsv)  # 將每個格子的平均HSV值儲存
        training_records[face] = cell_hsv_list# 將該面所有格子的HSV數據儲存到字典中
    return training_records

training_records = aggregate_all_cell_hsv(training_folder)
# 顯示所有面中各格子的 HSV 數值
for face, hsv_values in training_records.items():
    print(f"Face {face}:")
    for idx, hsv in enumerate(hsv_values):
        print(f"  Cell {idx + 1}: {hsv}")
folder_path = "/content/drive/MyDrive/numpy project/test/"  # 換成讀取test裡的資料
file_extensions = ('.pdf', '.png', '.jpg', '.jpeg')
test_files = sorted([os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.lower().endswith(file_extensions)])  # 讀取所有符合條件的檔案
images = []  # 用來儲存轉換後的影像

for file_path in test_files: # 讀取每個檔案並轉換RGB
    ext = os.path.splitext(file_path)[1].lower()
    if ext == ".pdf":
        img = pdf_to_cv2_image(file_path)
    else:
        img = cv2.imread(file_path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    images.append(img_rgb)  # 將處理後的影像加入列表

print(f"總共讀到 {len(images)} 張影像 (轉換自 PDF 或圖片)。")
#步驟1
def compute_face_cell_avgs(image):
    hsv_img = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
    h, w, _ = hsv_img.shape
    step_h = h // 3
    step_w = w // 3
    avgs = []
    for i in range(3):# 判斷從左到右、從上到下
        for j in range(3):
            square = hsv_img[i*step_h:(i+1)*step_h, j*step_w:(j+1)*step_w]#所有面的像素數據
            avg = np.mean(square.reshape(-1, 3), axis=0)#計算區塊的平均hsv值
            avgs.append(avg)
    return np.array(avgs)

# 計算所有54格的HSV平均值，並合併為陣列
cell_avgs_all = np.vstack([compute_face_cell_avgs(image) for image in images])
# 步驟2
color_order = ['U', 'R', 'F', 'D', 'L', 'B']
n_cells = cell_avgs_all.shape[0]#格子數量
n_colors = len(color_order)#顏色數量
cost = np.zeros((n_cells, n_colors))#儲存TEST與Train的相似度
for i in range(n_cells):
    for j, color in enumerate(color_order):
        #計算TEST與Train的最小歐式距離
        cost[i, j] = np.min(np.linalg.norm(cell_avgs_all[i] - training_records[color], axis=1))
#步驟3
fixed_indices = [4, 13, 22, 31, 40, 49]
fixed_labels = {4: 'U', 13: 'R', 22: 'F', 31: 'D', 40: 'L', 49: 'B'}
for idx in fixed_indices:
    expected = fixed_labels[idx]#預期顏色
    for j, color in enumerate(color_order):
        if color == expected:
            cost[idx, j] = 0
        else:
            cost[idx, j] = 1e6
#步驟4
#使用 Hungarian 演算法解分配問題
row_ind, col_ind = linear_sum_assignment(np.hstack([np.tile(cost[:, j:j+1], (1, 9)) for j in range(n_colors)]))
assignment = [color_order[col // 9] for col in col_ind]#將每個格子的顏色數據儲存到assignment裡面形成最終列表

# 步驟5
cube_faces_assigned = [assignment[i*9:(i+1)*9] for i in range(6)]
cube_string = "".join(sum(cube_faces_assigned, []))
print("Cube String:", cube_string)
print("各顏色出現次數:", Counter(cube_string))
import kociemba
print("魔術方塊的解法是：", kociemba.solve(cube_string))

魔術方塊的解法是： B D B D' B U R2 D' R2 D B2 U' B2
